In [88]:
import json
import os
import re
from rich.console import Console
from groq import Groq
from openai import OpenAI
from dotenv import load_dotenv

In [89]:
# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if groq_api_key:
    print(f"groq API Key exists and begins {groq_api_key[:7]}")
else:
    print("Groq API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
groq API Key exists and begins gsk_p3V


In [90]:
console = Console()

In [91]:
def get_current_weather(location: str, unit: str):
    """
    Get the current weather in a given location

    location (str): The city and state, e.g. Madrid, Barcelona
    unit (str): The unit. It can take two values; "celsius", "fahrenheit"
    """
    if location == "Madrid":
        return json.dumps({"temperature": 25, "unit": unit})

    else:
        return json.dumps({"temperature": 58, "unit": unit})

In [92]:
def add(num1, num2):
    return num1 + num2

In [93]:
def parse_tool_call_str(tool_call_str: str):
    # </? matches either a < or a </ (the ? makes the / optional). This matches the opening or closing angle bracket of an HTML tag.

    pattern = r"</?tool_call>"

    clean_tags = re.sub(pattern, "", tool_call_str)

    try:

        tool_call_json = json.loads(clean_tags)

        return tool_call_json

    except json.JSONDecodeError:

        return clean_tags

    except Exception as e:

        print(f"Unexpected error: {e}")

        return "There was some error parsing the Tool's output"

In [94]:
console.print(get_current_weather(location="Madrid", unit="celsius"))

{"temperature": 25, "unit": "celsius"}

In [95]:
LLM = "GROQ"
LLM = "OPENAI"

if LLM == "GROQ":
    client = Groq()
    MODEL = "llama3-groq-70b-8192-tool-use-preview"
if LLM == "OPENAI":
    client = OpenAI()
    MODEL = "gpt-4o-mini"

In [96]:
# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
<tools> {
    "name": "add",
    "description": "Adds two numbers and returns result",
    "parameters": {
        "properties": {
            "num1": {
                "type": "int"
            },
            "num2": {
                "type": "int"
            }
        }
    }
}
</tools>
"""

In [97]:
tool_chat_history = [{"role": "system", "content": TOOL_SYSTEM_PROMPT}]
agent_chat_history = []

user_msg = {
    "role": "user",
    # "content": "What is 10 plus 15 equal to?",
    "content": "What's the current temperature in Madrid, in Celsius?",
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = (
    client.chat.completions.create(messages=tool_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

print(output)

<tool_call>
{"name": "get_current_weather","arguments": {"location": "Madrid","unit": "celsius"}}
</tool_call>


In [98]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

In [99]:
if parsed_output["name"] == "add":
    num1 = parsed_output["arguments"]["num1"]
    num2 = parsed_output["arguments"]["num2"]

    result = add(num1, num2)
    console.print(
        f"[dark_orange bold]{parsed_output["name"]}[/] called with {num1} and {num2} with a result of [dark_orange]{result}[/]"
    )

In [100]:
if parsed_output["name"] == "get_current_weather":
    location = parsed_output["arguments"]["location"]
    unit = parsed_output["arguments"]["unit"]
    result = get_current_weather(location, unit)

In [101]:
agent_chat_history.append({"role": "user", "content": f"Observation: {result}"})

In [102]:
answer = (
    client.chat.completions.create(messages=agent_chat_history, model=MODEL)
    .choices[0]
    .message.content
)
console.print(f"[dark_orange bold]{answer}[/]")

The current temperature in Madrid is 25 degrees Celsius.